In [ ]:
from tqdm import tqdm
import pandas as pd
import fasttext
import numpy as np

In [ ]:
model = fasttext.load_model("/home/burak/Desktop/cc.en.300.bin")

In [ ]:
df = pd.read_csv("processed.csv",encoding="utf-8",lineterminator="\n")

In [ ]:
df

In [ ]:
sent_vectors = []
for item in tqdm(df.iterrows()):
    sent_vectors.append(model.get_sentence_vector(item[1].content))
    
sent_vectors = np.array(sent_vectors)

In [ ]:
labels_encoded = []
for item in tqdm(df.iterrows()):
    if item[1].label == "CYBER":
        labels_encoded.append(1)
    else:
        labels_encoded.append(0)
labels_encoded=np.array(labels_encoded)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(sent_vectors,labels_encoded,test_size=0.15)

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
score = clf.score(X_test, Y_test)
score

In [ ]:
import pickle
filename = 'svc.model'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
clf.predict([model.get_sentence_vector("today i've buy an asus laptop")])